<a href="https://colab.research.google.com/github/yukinaga/ai_webapp/blob/main/section_1/01_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最初のStreamlitアプリ
タイトルを表示するだけのシンプルなStreamlitアプリを作成し、ngrokを使って動作を確認します。

## ●ライブラリのインストール
Streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールします。

In [ ]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok==4.1.1 --quiet

Streamlit、およびngrokをインポートしておきます。  
エラーが発生する場合は、「ランタイム」→「ランタイムを再起動」によりランタイムを再起動し、再びコードセルを上から順に実行しましょう。

In [ ]:
import streamlit as st
from pyngrok import ngrok

## ●シンプルなアプリの作成
タイトルおよび表を表示するシンプルなアプリを作成し、ファイルに保存します。  
表の表示には、pandasのDataFrameを使います。  
`%%writefile`のマジックコマンドを使って、以下のコードを「app.py」に書き込みます。

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import numpy as np

# Interactive Streamlit elements, like these sliders, return their value.
# This gives you an extremely simple interaction model.
iterations = st.sidebar.slider("Level of detail", 2, 20, 10, 1)
separation = st.sidebar.slider("Separation", 0.7, 2.0, 0.7885)

# Non-interactive elements return a placeholder to their location
# in the app. Here we're storing progress_bar to update it later.
progress_bar = st.sidebar.progress(0)

# These two elements will be filled in later, so we create a placeholder
# for them using st.empty()
frame_text = st.sidebar.empty()
image = st.empty()

m, n, s = 960, 640, 400
x = np.linspace(-m / s, m / s, num=m).reshape((1, m))
y = np.linspace(-n / s, n / s, num=n).reshape((n, 1))

for frame_num, a in enumerate(np.linspace(0.0, 4 * np.pi, 100)):
    # Here were setting value for these two elements.
    progress_bar.progress(frame_num)
    frame_text.text("Frame %i/100" % (frame_num + 1))

    # Performing some fractal wizardry.
    c = separation * np.exp(1j * a)
    Z = np.tile(x, (n, 1)) + 1j * np.tile(y, (1, m))
    C = np.full((n, m), c)
    M = np.full((n, m), True, dtype=bool)
    N = np.zeros((n, m))

    for i in range(iterations):
        Z[M] = Z[M] * Z[M] + C[M]
        M[np.abs(Z) > 2] = False
        N[M] = i

    # Update the image placeholder by calling the image() function on it.
    image.image(1.0 - (N / N.max()), use_column_width=True)

# We clear elements by calling empty on them.
progress_bar.empty()
frame_text.empty()

# Streamlit widgets automatically run the script from top to bottom. Since
# this button is not connected to any other logic, it just causes a plain
# rerun.
st.button("Re-run")

## ●Authtokenの設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [ ]:
!ngrok authtoken YourAuthtoken

## ●アプリの起動と動作確認
streamlitの`run`コマンドでアプリを起動します。


In [ ]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [ ]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

urlを表示し、リンク先でアプリが動作することを確認します。

In [ ]:
print(url)